In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,accuracy_score,r2_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import numpy as np
from numpy import arange

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


**##Preliminary Data Anlaysis and Data cleaning###**

In [ ]:
##Reading the data from training set
df_train=pd.read_excel('Data_Train.xlsx')
df_train.tail()


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648
10682,Air India,9/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,10:55,19:15,8h 20m,2 stops,No info,11753


In [ ]:
##Reading the data from the test set
df_test=pd.read_excel('Test_set.xlsx')
df_test.tail()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info
2670,Multiple carriers,15/06/2019,Delhi,Cochin,DEL → BOM → COK,04:55,19:15,14h 20m,1 stop,No info


In [ ]:
##Combining the train and the test sets and using reset_index to ensure we have a consistent and sequential index
data=pd.concat([df_train,df_test],axis=0)
data.reset_index(drop=True)
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [ ]:
##Checking for the number of records and features
data.shape

(13354, 11)

In [ ]:
data.duplicated().sum()

246

In [ ]:
data[data.duplicated()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
683,Jet Airways,1/06/2019,Delhi,Cochin,DEL → NAG → BOM → COK,14:35,04:25 02 Jun,13h 50m,2 stops,No info,13376.0
1061,Air India,21/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 22 May,21h 15m,2 stops,No info,10231.0
1348,Air India,18/05/2019,Delhi,Cochin,DEL → HYD → BOM → COK,17:15,19:15 19 May,26h,2 stops,No info,12392.0
1418,Jet Airways,6/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 07 Jun,22h 55m,2 stops,In-flight meal not included,10368.0
1674,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,18:25,21:20,2h 55m,non-stop,No info,7303.0
...,...,...,...,...,...,...,...,...,...,...,...
2536,Air India,3/06/2019,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 04 Jun,21h 15m,2 stops,No info,NaN
2553,Jet Airways,15/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,19:00,13h 30m,2 stops,No info,NaN
2560,Multiple carriers,3/06/2019,Delhi,Cochin,DEL → BOM → COK,13:00,01:30 04 Jun,12h 30m,1 stop,No info,NaN
2582,Air India,12/05/2019,Kolkata,Banglore,CCU → BOM → GOI → BLR,16:50,05:35 13 May,12h 45m,2 stops,No info,NaN


In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
##Checking for missing values in each column
data.isnull().sum()

,0
Airline,0
Date_of_Journey,0
Source,0
Destination,0
Route,1
Dep_Time,0
Arrival_Time,0
Duration,0
Total_Stops,1
Additional_Info,0


In [ ]:
##Dropping Additional,Info and Route column as they do not offer much valuable insights to our price prediction
data.drop(columns=["Additional_Info"],inplace=True)
data.head()

In [ ]:
###Checking for the no of unique values in each column
data.nunique()

In [ ]:
###Dropping row where Route and Total stops is missing
data.dropna(subset=["Total_Stops"],axis=0,inplace=True)


In [ ]:
data.isnull().sum()

In [ ]:
# data["Day"]=data["Date_of_Journey"].str.split("/").str[0]
# data["Month"]=data["Date_of_Journey"].str.split("/").str[1]
# data["Year"]=data["Date_of_Journey"].str.split("/").str[2]



In [ ]:
data["Date_of_Journey"]=pd.to_datetime(data["Date_of_Journey"])
data["Day"]=data["Date_of_Journey"].dt.day
data["Month"]=data["Date_of_Journey"].dt.month
data["Year"]=data["Date_of_Journey"].dt.year
data.head()
# data["Day"]=data["Day"].apply(lambda x:x.split("/")[0])

In [ ]:
data["Day"]=data["Day"].astype(int)
data["Month"]=data["Month"].astype(int)
data["Year"]=data["Year"].astype(int)
data.info()

In [ ]:
data.drop("Date_of_Journey",axis=1,inplace=True)

In [ ]:
###Converting the duration column to minutes
def extract_time(time):
  if 'h' in time:
    hours=int(time.split('h')[0])
    if 'm' in time:
      minutes=int(time.split('h')[1].split('m')[0])
    else:
      minutes=0
  else:
        hours = 0
        if 'm' in time:
            minutes = int(time.split('m')[0])
        else:
            minutes = 0
  return hours * 60 + minutes

In [ ]:
data["Duration"]=data["Duration"].apply(lambda time: extract_time(time))

In [ ]:
data["Arrival_Hour"] = data["Arrival_Time"].str.split(" ").str[0].str.split(":").str[0].astype(int)
data["Arrival_Mins"] = data["Arrival_Time"].str.split(" ").str[0].str.split(":").str[1].astype(int)
# data["Arrival_Time"].unique()


In [ ]:
data.drop("Arrival_Time",axis=1,inplace=True)

In [ ]:
data["Dep_Hour"] = data["Dep_Time"].str.split(":").str[0].astype(int)
data["Dep_Mins"] = data["Dep_Time"].str.split(":").str[1].astype(int)

In [ ]:
data.drop("Dep_Time",axis=1,inplace=True)
data.head()

In [ ]:
data.info()

In [ ]:
data["Route"].unique()

array(['BLR → DEL', 'CCU → IXR → BBI → BLR', 'DEL → LKO → BOM → COK',
       'CCU → NAG → BLR', 'BLR → NAG → DEL', 'CCU → BLR',
       'BLR → BOM → DEL', 'DEL → BOM → COK', 'DEL → BLR → COK',
       'MAA → CCU', 'CCU → BOM → BLR', 'DEL → AMD → BOM → COK',
       'DEL → PNQ → COK', 'DEL → CCU → BOM → COK', 'BLR → COK → DEL',
       'DEL → IDR → BOM → COK', 'DEL → LKO → COK',
       'CCU → GAU → DEL → BLR', 'DEL → NAG → BOM → COK',
       'CCU → MAA → BLR', 'DEL → HYD → COK', 'CCU → HYD → BLR',
       'DEL → COK', 'CCU → DEL → BLR', 'BLR → BOM → AMD → DEL',
       'BOM → DEL → HYD', 'DEL → MAA → COK', 'BOM → HYD',
       'DEL → BHO → BOM → COK', 'DEL → JAI → BOM → COK',
       'DEL → ATQ → BOM → COK', 'DEL → JDH → BOM → COK',
       'CCU → BBI → BOM → BLR', 'BLR → MAA → DEL',
       'DEL → GOI → BOM → COK', 'DEL → BDQ → BOM → COK',
       'CCU → JAI → BOM → BLR', 'CCU → BBI → BLR', 'BLR → HYD → DEL',
       'DEL → TRV → COK', 'CCU → IXR → DEL → BLR',
       'DEL → IXU → BOM → COK', 'CCU 

In [ ]:
data=data[~data["Route"].isnull()]

In [ ]:
data["Route"].isnull().sum()

0

In [ ]:
##Splitting the Route column
data["Route1"]=data["Route"].str.split("→").str[0]
data["Route2"]=data["Route"].str.split("→").str[1]
data["Route3"]=data["Route"].str.split("→").str[2]
data["Route4"]=data["Route"].str.split("→").str[3]
data["Route5"]=data["Route"].str.split("→").str[4]
data["Route6"]=data["Route"].str.split("→").str[5]

In [ ]:
data["Route1"].fillna("None",inplace=True)
data["Route2"].fillna("None",inplace=True)
data["Route3"].fillna("None",inplace=True)
data["Route4"].fillna("None",inplace=True)
data["Route5"].fillna("None",inplace=True)
data["Route6"].fillna("None",inplace=True)

In [ ]:
data.drop(["Route"],axis=1,inplace=True)

**##Data Visualisation and further analysis###**

In [ ]:
###Airline feature analysis###
plt.figure(figsize = (15, 8))
ax=sns.countplot(data=data,x="Airline",order=data["Airline"].value_counts().index)
for label in ax.containers:
    ax.bar_label(label)
plt.xticks(rotation=85)
plt.tight_layout()
plt.show()

From the above visualization, it is found that JetAirways takes the first spot in terms of maximum flight operations followed by Indigo and Air India.Jet Airways Business,Vistara Premium ecocnomy and trujet have very minimal flights operating.

In [ ]:
###Source feature analysis###
plt.figure(figsize = (10, 8))
ax=sns.countplot(data=data,x="Source",order=data["Source"].value_counts().index)
for label in ax.containers:
    ax.bar_label(label)
plt.xticks(rotation=85)
plt.tight_layout()
plt.show()

Most of the flights (5681) take off from Delhi followed by Kolkata in the second place with 3581 flights and Bangalore in the third place with 2752 flights.Mumbai and Chennai occupy the last 2 spots.

In [ ]:
###Destination feature analysis###
plt.figure(figsize = (10, 8))
ax=sns.countplot(data=data,x="Destination",order=data["Destination"].value_counts().index)
for label in ax.containers:
    ax.bar_label(label)
plt.xticks(rotation=85)
plt.tight_layout()
plt.show()

Most of the passengers fly to the final destination Cochin,followed by Bangalore and Delhi in the second and third spots

In [ ]:
###Month feature analysis###
plt.figure(figsize = (10, 8))
ax=sns.countplot(data=data,x="Month",order=data["Month"].value_counts().index,hue=data["Airline"])
for label in ax.containers:
    ax.bar_label(label)
plt.xticks(rotation=85)
plt.tight_layout()
plt.show()
print(data["Month"].value_counts())

We can observe that the May and June months have many flights operating to various destinations as these months are generally summer vacation periods.JetAirways takes the first spot in opearting many flights in the months of March,May and June.On the otherhand GoAir,Vistara Premium Economy and JetAirways Business have very few flights operating in the these months.

In [ ]:
###Total Stops feature analysis###
plt.figure(figsize = (15, 10))
ax=sns.countplot(data=data,x="Airline", order=data["Airline"].value_counts().index,hue="Total_Stops")
for label in ax.containers:
    ax.bar_label(label)
plt.xticks(rotation=85)
plt.tight_layout()
plt.show()

Out of the 4746 flights operated by JetAirways, 3137 flights make atleast one stop before flying to the final destination.Indigo operates the highest number of non stop flights.AirIndia operates the most number of flights making 2 stops before reaching the final destination.It is also the only flight having 3 and 4 stops before reaching the final destination.

In [ ]:
bins=[0,2,4,6,8,10,12,14,16,18,20,22,23]
plt.hist(data["Dep_Hour"], bins=bins, edgecolor="yellow", color="green")
plt.xlabel("Departure hour")
plt.ylabel("Frequency")
plt.show()

Many flights depart between 6-10 AM and very few flights operate between 0-5 AM in the morning.

In [ ]:
plt.figure(figsize = (10,10))
ax=sns.barplot(x="Airline", y="Price", hue="Total_Stops", data=data)
plt.xlabel("Airline")
plt.ylabel("Price")
plt.xticks(rotation=85)
plt.title("Price of different airline by Total Stops")
plt.show()

1.Jet Airways Business flight with 1 stop has the highest ticket price which could possibly mean that the flight is flying to an international destaination and also the business class flights are generally expensive.

2.Indigo,SpiceJet,AirAsia and GoAir flights offer much lower fares for non-stops flights compared to other flights like Vistara,Vistara Premium Economy and JetAirways


In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(x=data["Day"], y=data["Price"], data=data)
plt.xlabel("Day of the Month")
plt.ylabel("Price")
plt.xticks(rotation=85)
plt.title("Price comparison by different days of the month")
plt.show()

Flight fares are on a higher side whent the travel date falls on the 1 st day of the Month while the fares seem to be lower during the last few days of the month

In [ ]:
bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12-14', '14-16', '16-18', '18-20', '20-22', '22-24']
data['Dep_Hour_Bin'] = pd.cut(data['Dep_Hour'], bins=bins, labels=labels)
mean_prices = data.groupby('Dep_Hour_Bin')['Price'].mean()
sns.barplot(x=mean_prices.index, y=mean_prices.values)
plt.xlabel("Departure Time")
plt.ylabel("Mean Price")
plt.xticks(rotation=85)
plt.title("Price comparison by departure time ")
plt.show()

Flights departing between 12-4 AM in the morning have the lowest fares  and the flights departing between 10PM-12AM have the highest fares

In [ ]:
# data.head()
# df=data.copy()
# price=df["Price"]
# df.drop("Price",axis=1,inplace=True)
# df=pd.concat([df,price],axis=1)
# df=df.reset_index(drop=True)
# df[df["Price"].isna()]


In [ ]:
###Dependent variable is not normally distributed and hence it needs to be scaled
sns.distplot(data["Price"],kde=True)

**##Feature Engineering##**

In [ ]:
##Encoding the values in the total stops column
data["Total_Stops"]=data["Total_Stops"].map({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4,'nan':1})

In [ ]:
##Display columns of Object data type
data.select_dtypes(['object']).columns

In [ ]:

# cat_data=pd.get_dummies(data[["Airline","Source","Destination"]],drop_first=True)
# cat_data.head()
# num_data=data.drop(["Dep_Hour_Bin","Airline","Source","Destination","Year"],axis=1)
# num_data.head()
# final_data=pd.concat([cat_data,num_data],axis=1)
# final_data=final_data.reset_index(drop=True)


In [ ]:
#Feature Encoding Airline,Source and Destination columns
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
data["Airline"]=encoder.fit_transform(data["Airline"])
data["Source"]=encoder.fit_transform(data["Source"])
data["Destination"]=encoder.fit_transform(data["Destination"])
data["Route1"]=encoder.fit_transform(data["Route1"])
data["Route2"]=encoder.fit_transform(data["Route2"])
data["Route3"]=encoder.fit_transform(data["Route3"])
data["Route4"]=encoder.fit_transform(data["Route4"])
data["Route5"]=encoder.fit_transform(data["Route5"])
data["Route6"]=encoder.fit_transform(data["Route6"])

In [ ]:
##Dropping the Dep Hour Bin column and segregating the train and the sets from the combined dataset
# df_train=final_data[0:10682]
# df_test=final_data[10682:]
# final_data.head()
df_train=data[0:10682]
df_test=data[10682:]
data.head()



In [ ]:
##Creating feature and target variables
X=df_train.drop("Price",axis=1)
test_data=df_test.drop("Price",axis=1)
y=df_train["Price"]
X.columns

In [ ]:
X.select_dtypes(include='number')

In [ ]:
###Perform feature selection and visualization
from sklearn.ensemble import ExtraTreesRegressor
selection= ExtraTreesRegressor()
selection.fit(X,y)
plt.figure(figsize = (12,8))
feat_importances = pd.Series(selection.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

Total Stops and Day of the Journey are considered to be relevant features in the prediction of the target variable y

In [ ]:
##Plotting the heatmap
C=final_data.corr()
plt.figure(figsize = (35, 19))
plt.rcParams.update({'font.size': 12})
sns.heatmap(C, annot = True, cmap = 'viridis', linewidths = 0.8)
plt.title('Correlation Heatmap', size = 35)
plt.show()

Duration and Total Stops features seem to be the most influential in the prediction of price

In [ ]:
##Splitting the dataset
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=0,shuffle=True)
##Applying Standardization to transform the numerical values to a common scale
scaler = StandardScaler().set_output(transform="pandas")
X_train_scaled=scaler.fit_transform(X_train.select_dtypes(include=np.number))
X_test_scaled=scaler.fit_transform(X_test.select_dtypes(include=np.number))

##Combine scaled data with encoded data
encoded_train_data=X_train.select_dtypes(exclude=np.number)
encoded_test_data=X_test.select_dtypes(exclude=np.number)
X_train_final=pd.concat([encoded_train_data,X_train_scaled],axis=1)
X_train_final=X_train_final.reset_index(drop=True)

X_test_final=pd.concat([encoded_test_data,X_test_scaled],axis=1)
X_test_final=X_test_final.reset_index(drop=True)

print(encoded_train_data.shape)
print(X_train_scaled.shape)
print(encoded_test_data.shape)
print(X_test_scaled.shape)








#




In [ ]:
###Define a function for evaluating the performance metrics

def perf_metrics(model):

  model.fit(X_train_final, y_train)

# Predict the test set labels
  y_pred = model.predict(X_test_final)

  y_pred_train=model.predict(X_train_final)

# Evaluate the test set RMSE
  r2_train=r2_score(y_train,y_pred_train)
  r2_test=r2_score(y_test,y_pred)
  mas=MAE(y_test,y_pred)
  mse=MSE(y_test,y_pred)
  rmse_test = np.sqrt(MSE(y_test, y_pred))
  return r2_train,r2_test,mas,mse,rmse_test





In [ ]:
##Random Forest Regressor
rf = RandomForestRegressor(n_estimators=50,max_depth=10,random_state=42,oob_score=True)
r2_train,r2_test,mas,mse,rmse_test=perf_metrics(rf)

print("Train score:",r2_train)
print("Test score:",r2_test)
print("Mean absolute error:",mas)
print("Mean squared error:",mse)
print("Root mean squared error:",rmse_test)

##OOB Accuracy (out of bag accuracy which estimates model's generalisation performance)
acc_oob = rf.oob_score_

##Printing the test accuracy and OOB accuracy
print('OOB accuracy: {:.3f}'.format(acc_oob))


# Create a pd.Series of features importances
importances = pd.Series(data=rf.feature_importances_,
                        index= X_train_final.columns)

# Sort importances
importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
importances_sorted.plot(kind='barh', color='purple')
plt.title('Features Importances')
plt.show()

Overall, these metrics suggest that our Random Forest model is performing reasonably well, with a good R² score and OOB accuracy.

In [ ]:
##SVR Regressor
from sklearn.svm import SVR
##C is like a penalty term (regualarisation parameter to prevent model overfitting and gamma parameter controls the complexity of model)
svr_model = SVR(kernel ='rbf',C=10000,gamma=0.1)
r2_train,r2_test,mas,mse,rmse_test=perf_metrics(svr_model)
print("Train score:",r2_train)
print("Test score:",r2_test)
print("Mean absolute error:",mas)
print("Mean squared error:",mse)
print("Root mean squared error:",rmse_test)



The model explains about 77.84% of variance in the training data.It is a decent score but not a high score.Also, the model explains about 72.07% of the variance in the testing data. This is slightly lower than the training score, which is expected, as the model may not generalize as well to new data.The MAE is around 1348.97 which means that the model is, on average, off by about 1348.97 units from the actual values.

In [ ]:
###GradientBoostingRegressor
grb=GradientBoostingRegressor(learning_rate=0.1,n_estimators=90, max_depth=5,verbose=False,random_state=10)
r2_train,r2_test,mas,mse,rmse_test=perf_metrics(grb)
print("Train score:",r2_train)
print("Test score:",r2_test)
print("Mean absolute error:",mas)
print("Mean squared error:",mse)
print("Root mean squared error:",rmse_test)

Gradient Boosting Regressor is performing reasonably well, but there is still room for improvement. The model is fitting the training data well, but it may be overfitting slightly, as the test score is lower than the train score.The MAE and RMSE indicate that the model is making errors of around 1346.50 and 2044.47 units, respectively.

In [ ]:
##CatBoost Regressor
!pip install catboost
import catboost as cb
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100,learning_rate=0.25,depth=6)
r2_train,r2_test,mas,mse,rmse_test=perf_metrics(model)
print("Train score:",r2_train)
print("Test score:",r2_test)
print("Mean absolute error:",mas)
print("Mean squared error:",mse)
print("Root mean squared error:",rmse_test)

The model has performed quite well but the test score seems to be lower than the training score which indicates slight overfitting. On average, the model's predictions are off by about 1336.09 units from the actual values.

In [ ]:
# Create a pd.Series of features importances
# importances = pd.Series(data=rf.feature_importances_,
#                         index= X_train_final.columns)

# Sort importances
# importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
# importances_sorted.plot(kind='barh', color='purple')
# plt.title('Features Importances')
# plt.show()